In [3]:
import pandas as pd
import numpy as np
from datetime import timedelta
import pickle
from numpy import median
import lightgbm as lgb
import os

In [47]:
client = pd.read_csv('example_test_files/client.csv', keep_default_na=False)
forecast = pd.read_csv('example_test_files/forecast_weather.csv', keep_default_na=False)
hist = pd.read_csv('example_test_files/historical_weather.csv', keep_default_na=False)
gas = pd.read_csv('example_test_files/gas_prices.csv', keep_default_na=False)
elec = pd.read_csv('example_test_files/electricity_prices.csv', keep_default_na=False)
test = pd.read_csv('example_test_files/test.csv', keep_default_na=False)
calendar = pd.read_csv('calendar.csv', keep_default_na=False)
gps_point = pd.read_csv('data/gps_point.csv', keep_default_na=False)
code_list = pd.read_csv('code_list.csv', keep_default_na=False)

In [48]:
forecast['lat/long'] = forecast['latitude'].astype(str) + ',' + forecast['longitude'].astype(str)
selected_gps = gps_point.loc[gps_point['main_gps_county'] == 1, 'lat/long'].tolist()
forecast = forecast[forecast['lat/long'].isin(selected_gps)]
forecast = pd.merge(forecast, gps_point[['lat/long', 'county_code']], on='lat/long', how='left')
forecast['county_code'] = forecast['county_code'].astype('int64')

In [6]:
test['prediction_datetime'] = pd.to_datetime(test['prediction_datetime'], errors='coerce')
test['datetime_date'] = test['prediction_datetime'].dt.date
test['datetime_date'] = pd.to_datetime(test['datetime_date'])
test['datetime_time'] = test['prediction_datetime'].dt.time
test['datetime_time'] = pd.to_datetime(test['datetime_time'], format='%H:%M:%S').dt.hour.astype('int64')
# test['datetime_time'] = test['datetime_time'].astype('int64')
test['year'] = test['prediction_datetime'].dt.year
test['month'] = test['prediction_datetime'].dt.month
test['week_num'] = test['prediction_datetime'].dt.isocalendar().week
test['week_num'] = test['week_num'].astype('int64')
test['day'] = test['prediction_datetime'].dt.day

In [7]:
gas['mwh_mean'] = gas[['lowest_price_per_mwh', 'highest_price_per_mwh']].mean(axis=1)

In [8]:
def convert_dates(df):
    for column in df.columns:
        if column.lower().startswith('date') or '_date' in column.lower():
            df[column] = pd.to_datetime(df[column], errors='coerce')
    return df

# Apply date conversion to each DataFrame
client = convert_dates(client)
forecast = convert_dates(forecast)
hist = convert_dates(hist)
gas = convert_dates(gas)
elec = convert_dates(elec)
test = convert_dates(test)
calendar = convert_dates(calendar)
gps_point = convert_dates(gps_point)

In [9]:
def add_days_to_dates(df, date_column, days):
    df[date_column] = df[date_column] + pd.Timedelta(days=days)

# move calendar dates of client, gas, elec
add_days_to_dates(client, 'date', 2)
add_days_to_dates(gas, 'forecast_date', 1)
add_days_to_dates(elec, 'forecast_date', 1)

In [10]:
def generate_code(row):
    return (row['county'] * 7) + (row['product_type'] * 2) + row['is_business']

# add code column to test and client
client['code'] = client.apply(generate_code, axis=1)
test['code'] = test.apply(generate_code, axis=1)

In [11]:
median_val = elec['euros_per_mwh'].median()
mad_val = elec['euros_per_mwh'].mad()
elec['z_score'] = 0.6745 * ((elec['euros_per_mwh'] - median_val) / mad_val)

threshold = 6
outliers = abs(elec['z_score']) > threshold
elec.loc[outliers, 'euros_per_mwh'] = np.nan
elec['euros_per_mwh'] = elec['euros_per_mwh'].fillna(method='ffill')

Merge

In [12]:

print("Data types before merge:")
print(test[['datetime_date']].dtypes)
print(gas[['forecast_date']].dtypes)
    
test = test.merge(elec, left_on='prediction_datetime', right_on='forecast_date', how='left', suffixes=('', '_elec'))
test = test.merge(gas, left_on='datetime_date', right_on='forecast_date', how='left', suffixes=('', '_gas'))
test = pd.merge(test, client[['date', 'code', 'eic_count', 'installed_capacity']],
                    left_on=['datetime_date', 'code'],
                    right_on=['date', 'code'],
                    how='left',
                    suffixes=('_test_solar_', '_client'))
    
print("Data types and columns:")
print(test.columns)
print(forecast.columns)
test = pd.merge(test, forecast,
                    left_on=['prediction_datetime', 'county'],
                    right_on=['forecast_datetime', 'county_code'],
                    how='left',
                    suffixes=('_forecast_weather_', '_forecast_weather2_'))
    
test = pd.merge(test, calendar[['date', 'working_day']], left_on='datetime_date', right_on='date', how='left')

Data types before merge:
datetime_date    datetime64[ns]
dtype: object
forecast_date    datetime64[ns]
dtype: object
Data types and columns:
Index(['county', 'is_business', 'product_type', 'is_consumption',
       'prediction_datetime', 'data_block_id', 'row_id', 'prediction_unit_id',
       'currently_scored', 'datetime_date', 'datetime_time', 'year', 'month',
       'week_num', 'day', 'code', 'forecast_date', 'euros_per_mwh',
       'origin_date', 'data_block_id_elec', 'z_score', 'forecast_date_gas',
       'lowest_price_per_mwh', 'highest_price_per_mwh', 'origin_date_gas',
       'data_block_id_gas', 'mwh_mean', 'date', 'eic_count',
       'installed_capacity'],
      dtype='object')
Index(['latitude', 'longitude', 'origin_datetime', 'hours_ahead',
       'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low',
       'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component',
       '10_metre_v_wind_component', 'data_block_id', 'forecast_datetime',
       'direct_sola

In [13]:
models_folder='models'

cd_list = code_list['code'].to_list()

# Iterate over rows in the 'test' DataFrame
for index, row in test.iterrows():
        # Check if 'is_consumption' is 0
        if row['is_consumption'] == 0:
            test.at[index, 'model'] = os.path.join(models_folder, 'lightgbm_model.pkl')
        else:
            # Check if 'code' is in the code list
            if row['code'] in cd_list:
                test.at[index, 'model'] = os.path.join(models_folder, f'model_{row["code"]}.pkl')

In [14]:
# reduce the columns of test to specified columns
cols = ['row_id', 'prediction_datetime', 'datetime_time', 'week_num', 'working_day',
            'county', 'is_business', 'product_type', 'is_consumption', 'code',
            'prediction_unit_id', 'euros_per_mwh', 'mwh_mean', 'eic_count',
            'installed_capacity', 'temperature', 'dewpoint', 'snowfall',
            'cloudcover_total', 'cloudcover_low',
            'cloudcover_mid', 'cloudcover_high', 'surface_solar_radiation_downwards', 'year', 'month', 'day', 'model']

test = test[cols]
    # reset the type of eic_count back to float
test['eic_count'] = test['eic_count'].astype(float)
test['datetime_time'] = test['datetime_time'].astype('int64')

In [23]:
pd.set_option('display.max_columns', 30)
test.head()

,row_id,prediction_datetime,datetime_time,week_num,working_day,county,is_business,product_type,is_consumption,code,prediction_unit_id,euros_per_mwh,mwh_mean,eic_count,installed_capacity,temperature,dewpoint,snowfall,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,surface_solar_radiation_downwards,year,month,day,model
0,2005872,2023-05-28,0,21,0,0,0,1,0,2,0,87.54,31.2,507.0,4960.215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,5,28,models\lightgbm_model.pkl
1,2005873,2023-05-28,0,21,0,0,0,1,1,2,0,87.54,31.2,507.0,4960.215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,5,28,models\model_2.pkl
2,2005874,2023-05-28,0,21,0,0,0,2,0,4,1,87.54,31.2,11.0,34.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,5,28,models\lightgbm_model.pkl
3,2005875,2023-05-28,0,21,0,0,0,2,1,4,1,87.54,31.2,11.0,34.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,5,28,models\model_4.pkl
4,2005876,2023-05-28,0,21,0,0,0,3,0,6,2,87.54,31.2,1516.0,15977.560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,5,28,models\lightgbm_model.pkl


In [27]:
pd.set_option('display.max_columns', 30)
forecast.head()

,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation,lat/long,county_code


Predict

In [ ]:
# Load the LightGBM model
model_path_lgb = 'models/lightgbm_model.pkl'  # Adjust the path accordingly
with open(model_path_lgb, 'rb') as model_file:
    lightgbm_model = pickle.load(model_file)

    submission_test = pd.DataFrame(columns=['row_id', 'target'])
    
    # Iterate over rows in the 'test' DataFrame
    for index, row in test.iterrows():
        # Check the value in 'model' column
        if row['model'] == os.path.join('models', 'lightgbm_model.pkl'):
            # Use LightGBM prediction
            data = row.drop(['row_id', 'prediction_datetime', 'model'])
            prediction = lightgbm_model.predict(data.values.reshape(1, -1))
        else:
            # Load the Linear Regression model indicated in 'model' column
            with open(row['model'], 'rb') as model_file:
                lr_model = pickle.load(model_file)
            
            # Use Linear Regression prediction
            data = np.array(row['temperature']).reshape(1, -1)
            prediction = lr_model.predict(data)
        
        # Append the prediction to the submission DataFrame
        submission_test = submission_test.append({'row_id': int(row['row_id']), 'target': prediction[0]}, ignore_index=True)

    # Convert 'row_id' column to int64 explicitly
    submission_test['row_id'] = submission_test['row_id'].astype('int64')

In [50]:
submission_test.head()

NameError: name 'submission_test' is not defined